# Visualization

In this notebook we are going to visualize interesting results

In [140]:
import pandas as pd
import bokeh
from folium import plugins
import folium
import numpy as np
import math
import os
#import warnings
#warnings.filterwarnings('ignore')

# Preprocess

In [141]:
def createHeatMap():
    ## HeatMap
    heatmap = folium.Map(location=[np.mean(lats), np.mean(lons)], zoom_start=8, tiles = 'cartodbdark_matter',
               min_zoom=8,min_lat=45,max_lat=48,min_lon=5,max_lon=11)
    
    return heatmap

def createHeatMapLayer(df_array, name_array, heatmap):
    for df,name in zip(df_array, name_array):
        lats = df['Latitude'].values
        lons = df['Longitude'].values

        heatmapLayer1 = plugins.HeatMap(zip(lats, lons), radius = 10,min_opacity=0.7,blur=15)
        heatmap.add_children(heatmapLayer1)

        heatmapLayer1.layer_name = name
    
    folium.LayerControl().add_to(heatmap)
    
    return heatmap
    
def createDetailedMap():
    ## DetailedMap
    map_detailed = folium.Map(location=[np.mean(lats), np.mean(lons)], zoom_start=8, tiles = 'cartodbdark_matter',
               min_zoom=8,min_lat=45,max_lat=48,min_lon=5,max_lon=11)

    return map_detailed
    
def createDetailedMapLayer(df_array, name_array, map_detailed):
    
    for df,name in zip(df_array, name_array):
        df_grouped = df.groupby(['Venue'])
        aggregation = {'Latitude':'mean',
                   'Longitude':'mean',
                   'Artist':'count'
                  }
        df_p = df_grouped.agg(aggregation)
        popups, locations = [], []


        marker_clusterLayer = folium.MarkerCluster()
        marker_clusterLayer.layer_name = name
        
        map_detailed.add_children(marker_clusterLayer)
        #marker_clusterLayer.add_to(map_detailed)
        #Marker creation
        for i in range(len(df_p)):
            popuptext = df_p.ix[i].name+' - '+str(math.floor(df_p.ix[i].Artist))+' artists'
            folium.CircleMarker(location=[df_p.ix[i].Latitude, df_p.ix[i].Longitude], 
                                radius=5,
                            popup=popuptext, color='#3186cc',
                            fill_color='#3186cc').add_to(marker_clusterLayer)
    
    folium.LayerControl().add_to(map_detailed)

    return map_detailed


In [142]:
total_events = pd.read_csv(os.path.join('./FinalResults/total_eventsFinal.csv'))
missing_coord = sum(pd.isnull(total_events['Latitude']))
missing_coord_percentage = round(100*missing_coord/len(total_events),3)
print('There are ',missing_coord, ' missing venues coordinates in the dataset, which represent ',
      missing_coord_percentage, '% of the total')

total_events.dropna(inplace = True)
total_events.drop(['Date','City'], axis = 1, inplace = True)
folderData = 'FoliumData'
folderResults = 'FinalResults'

## HeatMap
heatmap = createHeatMap()
heatMap_all = createHeatMapLayer([total_events], ['2006 - 2016 / All Genres'], heatmap)


## DetailedMap
detailedMap = createDetailedMap()
detailedMap_all = createDetailedMapLayer([total_events], ['2006 - 2016 / All Genres'], detailedMap)


There are  84  missing venues coordinates in the dataset, which represent  0.041 % of the total


# For all years, all genres

## Heat Map

This map shows the number of concerts. The more red, the higher number of concerts happenned. It allows to immediatley see the areas with a lot of musical events.

In [100]:
destinationFileName = os.path.join(folderResults, 'heatMap_all.html')    
heatMap_all.save(destinationFileName)
heatMap_all

## Detailed Map
This map shows the number of concerts. The bigger the circle, the higher number of concerts happened. This map is interractive, we can click on the circle to obtain the name of the venue and the number of concerts.

In [101]:
destinationFileName = os.path.join(folderResults, 'detailedMap_all.html')    
detailedMap_all.save(destinationFileName)
detailedMap_all

## Per genre, for all years

In [102]:
# Just for test, to be deleted later:
df1 = total_events[:500] 
df2 =  total_events[500:] 
df1['genre']= 'Rock'
df2['genre']= 'Electro'

/Users/peco/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/peco/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [104]:
## HeatMap
heatmap2 = createHeatMap()
heatMap_all = createHeatMapLayer([df1, df2], ['2006 - 2016 / Rock', '2006 - 2016 / Electro'], heatmap2)

heatMap_all

In [107]:
detailedMap2 = createDetailedMap()
detailedMap_all = createDetailedMapLayer([df1, df2], ['2006 - 2016 / Rock', '2006 - 2016 / Electro'], detailedMap2)
detailedMap_all